In [1]:
import re
import nltk
import numpy as np

In [2]:
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [3]:
np.random.seed(7)
nb_word_class = 5

## Tag the training data

In [4]:
def clean_sentence(s):
    c = s.lower().strip()
    return re.sub('[^a-z ]', '', c)

In [5]:
def translate_word_class(tag):
    if tag in ['JJ', 'JJR', 'JJS']:
        return 1 #'adjective'
    if tag in ['NN', 'NNS', 'NNP', 'NNPS']:
        return 2 #'noun'
    if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return 3 #'verb'
    if tag in ['CC', 'IN']:
        return 4 #'link'
    return 0 #'other'

In [6]:
def map_tagging(sentence):
    translation = list()
    for word, tag in sentence:
        translation.append((word, translate_word_class(tag)))
    return translation

In [7]:
def map_encoding(sentence):
    encoded = list()
    for word, tag in sentence:
        encodedInt = one_hot(word,30000)[0]
        encoded.append([encodedInt,tag])
    return encoded        

In [8]:
END = 6
def sentence_labeling(sentence):
    labels = list()
    for word, tag in sentence[1:]:
        labels.append(tag)
    labels.append(END)
    return labels

In [9]:
def tag_words(sentences):
    tagged_words = list()
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged_words.append(nltk.pos_tag(words))
    return tagged_words

In [10]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [11]:
def create_Ymatrix(numbers,size,var_index):
    matrix = np.zeros((size))
    i = 0
    for number in numbers:
        matrix[i] = number[var_index]
        i=i+1
    return matrix

In [12]:
def create_Xmatrix(numbers,size,var_index):
    matrix = np.zeros((size,1))
    sub_matrix = np.zeros((1))
    i = 0
    for number in numbers:
        sub_matrix[0] = number[var_index]
        matrix[i][0] = sub_matrix
        i=i+1
    return matrix

In [13]:
train_sentences = list()
with open('./data_set/training_set70.txt') as train:
    for line in train:
        train_sentences.append(clean_sentence(line))

test_sentences = list()
with open('./data_set/test_set25.txt') as train:
    for line in train:
        test_sentences.append(clean_sentence(line))

In [14]:
train_sentences[:5]

['correct', 'dick agreed', 'sorry wrong', 'where is mary', 'he is happy now']

In [15]:
tagged_sentences = tag_words(train_sentences)
tagged_sentences_test = tag_words(test_sentences)

In [16]:
tagged_sentences[:5]
tagged_sentences_test[:5]

[[('sam', 'JJ'), ('didnt', 'NNS'), ('like', 'IN'), ('elections', 'NNS')],
 [('dick', 'NN'),
  ('and', 'CC'),
  ('sam', 'NN'),
  ('were', 'VBD'),
  ('disappointed', 'JJ')],
 [('where', 'WRB'), ('is', 'VBZ'), ('my', 'PRP$'), ('apple', 'NN')],
 [('yes', 'RB'), ('thats', 'NNS'), ('correct', 'VBP')],
 [('no', 'DT'), ('its', 'PRP$'), ('wrong', 'NN')]]

In [17]:
my_tagged_sentences = list(map(map_tagging, tagged_sentences))
test_tagged_sentences = list(map(map_tagging, tagged_sentences_test))

In [18]:
my_tagged_sentences[:5]

[[('correct', 2)],
 [('dick', 2), ('agreed', 3)],
 [('sorry', 2), ('wrong', 1)],
 [('where', 0), ('is', 3), ('mary', 1)],
 [('he', 0), ('is', 3), ('happy', 1), ('now', 0)]]

In [19]:
x = flatten(map(map_encoding, my_tagged_sentences))
test = flatten(map(map_encoding, test_tagged_sentences))


In [20]:
x_train = create_Xmatrix(x,len(x),0)
"y_train = create_Ymatrix(x,len(x),1)"
y_train = to_categorical(create_Ymatrix(x,len(x),1),nb_word_class)
x_test = create_Xmatrix(test,len(test),0)
y_test = to_categorical(create_Ymatrix(test,len(test),1),nb_word_class)

print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(y_train)
print(x_test)

X_train shape: (328, 1)
X_test shape: (132, 1)
y_train shape: (328, 5)
y_test shape: (132, 5)
[[ 0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 ..., 
 [ 0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]]
[[  2.64960000e+04]
 [  4.15300000e+03]
 [  2.30400000e+04]
 [  1.51660000e+04]
 [  8.65200000e+03]
 [  1.89720000e+04]
 [  2.64960000e+04]
 [  7.28100000e+03]
 [  1.53210000e+04]
 [  8.49900000e+03]
 [  2.75040000e+04]
 [  1.75770000e+04]
 [  2.21400000e+04]
 [  2.72070000e+04]
 [  5.60800000e+03]
 [  2.80290000e+04]
 [  1.07830000e+04]
 [  2.04300000e+03]
 [  4.54800000e+03]
 [  7.00400000e+03]
 [  2.25200000e+03]
 [  5.40300000e+03]
 [  8.04800000e+03]
 [  2.24840000e+04]
 [  2.01690000e+04]
 [  2.26850000e+04]
 [  7.52500000e+03]
 [  8.22000000e+02]
 [  4.15300000e+03]
 [  2.94270000e+04]
 [  2.01690000e+04]
 [  2.06150000e+04]
 [  2.01690000e+04]
 [  2.23950000e+04]
 [  1.10110000e+04]
 [  2.65100000e+04]
 [  2.90600000e+03]
 [  2.75040000e+04]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/keras/utils/np_utils.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


In [22]:
print('Build model...')
model = Sequential()
model.add(Embedding(30000,256, dropout=0.2))
model.add(LSTM(8, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(nb_word_class))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train, batch_size=328, nb_epoch=10,
         validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=328)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 328 samples, validate on 132 samples
Epoch 1/10
328/328 [==============================] - 1s - loss: 1.6089 - acc: 0.2683 - val_loss: 1.6078 - val_acc: 0.3864
Epoch 2/10
328/328 [==============================] - 0s - loss: 1.6073 - acc: 0.3720 - val_loss: 1.6060 - val_acc: 0.4545
Epoch 3/10
328/328 [==============================] - 0s - loss: 1.6042 - acc: 0.5091 - val_loss: 1.6042 - val_acc: 0.5152
Epoch 4/10
328/328 [==============================] - 0s - loss: 1.6012 - acc: 0.5854 - val_loss: 1.6022 - val_acc: 0.5833
Epoch 5/10
328/328 [==============================] - 0s - loss: 1.5989 - acc: 0.6250 - val_loss: 1.6003 - val_acc: 0.6288
Epoch 6/10
328/328 [==============================] - 0s - loss: 1.5970 - acc: 0.6524 - val_loss: 1.5984 - val_acc: 0.6591
Epoch 7/10
328/328 [==============================] - 0s - loss: 1.5937 - acc: 0.7287 - val_loss: 1.5965 - val_acc: 0.6742
Epoch 8/10
328/328 [==============================] - 0s - loss: 1.5910 - acc: 0.7439 - val_l